<a href="https://colab.research.google.com/github/gixhswlab1ppm/ppm/blob/master/Homework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 4: Final Report Part 1
This homework is very different from previous homeworks. The most signficant change is that this is a __group assignment__ that focuses on your project. You should complete it in collaboration with members of your project group. There is no coding, so you should feel free to complete the assignment in word or whatever format you are most comfortable with. This will be worth a significant amount of your final grade, so please try to do a good job. 

As usual, the homework is broken into tasks, however, each task is a phase of planning. For each task, you're highly encouraged to include graphics that aid in your explanations and thought processes. If you need help making these graphics, I recommend taking a look at [yed](https://www.yworks.com/products/yed), a really neat graph editor.

Only one member of the group should submit this assignment, however, please provide the names of each member of the group here:

**Group Members:**
*   James Gan
*   Jason Cui
*   Shine Lin



## Task 1: Problem Description
For this task you'll be describing what you want to accomplish. Of course some of this can be copy pasted from previous presentations you've done, but there are additional considerations you'll have to make. Specifically you should:
* Describe the goal of your project

Table tennis is a popular Olympic sport with very few professional players in most countries. It is expensive and can be difficult to learn with low availability of ways to improve. As a result, our overall goal is to build an electronic device that enables an efficient learning experience for table tennis. The device should be capable of detecting data from a collection of sensors, processing that data with standard computation and pre-trained models for real-time analysis, and to send that data to the cloud for more complex machine learning analysis,

* What requirements does the project have? For example, what accuracy do you need to create a compelling experience? How fast does it need to run? Are there any privacy concerns? Etc...

Our project does not have extreme accuracy requirements: we are targeting beginners, so anything that has a high degree of accuracy towards suggesting the right direction (aka. binary classification) of change for improvement will be sufficient for our design. As a result, we **do want** a high accuracy in the direction of our outputs (e.g. was that game good or bad) but **do not need** high accuracy for more precise purposes (e.g. change the vertical acceleration of the swing 1m/s in order to vastly improve).

For any pre-trained model we want to put onto our device's on-board Raspberry Pi, we will need it to run very fast: we want it to display results in near-real-time such that players would be able to seamlessly analyze their plays. However, for any models that are not fast, we would put them on the cloud for **post match analysis** which would arguably be fine even if they had up to an hour of delay. **We do not have privacy concerns**: all the data collected is not particularly sensitive - we do not have any biometric or otherwise personally identifiable information.

* What class of machine learning problem is this? Is there overlap between several classes? In this context, a class of problem would be simple classification, natural language processing, vision, or one of the other topics discussed.

Our project requires a mix of machine learning and data mining algorithms to fulfill the following tasks:

(1) Classify if a player's swing is good or bad by looking at the swing movement before hitting a ball

<i>Classification, Clustering</i>

(2) Predict the current player by looking at the swing movement

<i>Classification, Clustering</i>

(3) Predict the quality scores of hitting by looking at the swing movement before hitting a ball

<i>Regression</i>


## Task 2: Dataset Description
In this task, you'll consider the dataset being collected to achieve your goal.
* Which features do you need to collect? How do each of these features relate to the output of your network?

Raw input features: IMU acceleration/IMU angular velocity ($F^6$), piezoelectric disk reading/vibration ($N^3$)

Manually labelled features: Player Quality ($B$), Player Id ($N$)

Engineering features: Ball Hitting Event ($N$), Ball Hitting Temporal Dispersity/Ball Hitting Spatial Dispersity/Ball Hitting Strength($F^3\times N$), Top-K Signal In Frequency Domain ($F\times N$)

Engineering features are features that are generated from raw input through certain functions. Ultimately, a mix of any kind of features are *directly* fed into the network for training. The network output contains Player Quality, Player Id, BHTD/BHSD/BHS only.

* Describe your data collection procedure. How will you store the data? How will you prevent accidentally teaching the network something you didn't mean to. For example, If I trained a network with images all from the same room, it might learn to look for that background and not work anywhere else.

Given the real-time stream nature of sensor data and batch-processed near-real-time nature of enginnering features, the data is stored in a circular buffer residing in main memory first, then "dumped"/serialized to JSON files (when a threshold of # of pieces of data is reached) and saved both locally and to the cloud.

To prevent overfitting during data collection, we ask testers to "repeat swinging the paddle and hitting the ball" at their convenience, which would create a rich dataset that differs in the frequency of swings, the direction/angle of swings (which slightly affects gyroscope), quality of hits, etc. Since we plan to only collect a relatively small amount of data, we do not have to select testers based on ping-pong players' statistical distribution.

* How many samples do you think you'll need? This relates both to the difficulty of the problem and the complexity of the network. How long do you expect it'll take to collect those samples? How much memory do you expect your dataset to take?

We would need 10 samples each contains at least 15 "swing & hit" pairs. This gives as 150 "swing & hit" pairs in total, and each pair will be the data collected from sensors during ~30 seconds. Given the current sample rate of 5 data points/sec and 40 Bytes per data point, each pair will take 6KB. The dataset will be at least 9MB in memory and probably 2x that size in persist media.

We would need to ask 10 different players to play around 1 minute each, which takes 10 minutes in theory. If taking experiment setting-up and post-processing time into consideration, it would take us 30 minutes at least.


* What are the labels for your data? How are they being labeled? Do you think there's any bias that might be accidentally inserted?

Good/Bad player. Subjectively labelled by three of us - each casting a vote for good/bad, and using the popular vote as the label. Bias may definitely be inserted given our subjective/non-quantitative approach to estimating a player's performance. Plus, personal relationship (e.g. giving your good friend a "Good Player" label) may also result in bias.


## Task 3: Model Design
Now that we have our goal and dataset set up, let's think about how to build a model that achieves our goal. For this task, you absolutely should include a graphic that shows the structure of your proposed architecture.
* What type of architecture will you use? What are the pros and cons of your choice? Are there any alternative architectures that might work too?
* Show the structure of your proposed architecture (graphic here)
* Is there anything special about your architecture, such as a structural difference not covered in class?

## Task 4: Training
Let's now consider how you'll be training the model described. Here, consider how difficult the model might be train and what special considerations you may need to take for your project.
* What hyperparameters make sense for your project? Hyperparameters include:
    * Optimization algorithm (sgd, adam, rmsprop, etc)
    * Learning rate
    * Training time (epochs)
    * Regularization (weight decay)
* Are there any special training considerations you need to make? For example, reinforcement learning has a very different flow from normal training. What are those adjustments?
* How will you evaluate the progress of your training. Do you think your validation accuracy will reflect real life?

## Task 5: Deployment
Finally, you have a trained model that accomplishes your initial goals. Now, consider how you'll take this model and make something useful from it.
* Where will the model be run? Does it make sense to have it running on an Azure server? Why or why not? Does it make sense to have it running locally (on raspberry pi for example)? Why or why not?
* How many users do you think could use your product at the same time? Will it effect performance?
* How will you test how well your final product works?